In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performerErrorTypeTest_V2 as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [3]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return tokens

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [10]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce-binary-accuracy"
        Input_Path = "Trianing\InputTxt\Split-500-reduce-binary-accuracy"
        model_for_training_org_path = "Model-for-training-org\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\weight-1vs1-V2"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        #get all txt file in input path
        target_max_len = 0
        token_num = 0

        #start training
        import DataGeneratorTrainErrorTypeEdition as DGTrain
        import DataGeneratorValidationErrorTypeEdition as DGValidation
        import DataBuffer as db
        from random import randrange
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #load source_max_len
        source_max_len = loadDictionary(Trained_model_Path + "/" + source_max_len_name)
        #load training models and their len
        x_train = loadTestTrainData(model_for_training_path + "/" + "x_train_0.npy")
        x_validation = loadTestTrainData(model_for_training_path + "/" + "x_validation_0.npy")
        
        buffer_train_num = len(x_train)
        buffer_val_train_num = len(x_validation)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              errNum=36,
                              token_num=len(source_token_dict),
                              embed_dim=32, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        
        #for output
        #for x
        #data_number and block_size = data num
        input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_train_num,
            "data_type": int,
            "block_size": buffer_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = data num
        """
        output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_train_num, buffer_train_num],
            "data_type": [int, int],
            "block_size": [buffer_train_num, buffer_train_num] 
            }
        """
        output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_train_num],
            "data_type": [int],
            "block_size": [buffer_train_num] 
            }
        
        #===========================================
        #for output
        #for x
        #data_number and block_size = validation data num
        
        validation_input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_val_train_num,
            "data_type": int,
            "block_size": buffer_val_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = validation data num
        """
        validation_output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_val_train_num, buffer_val_train_num],
            "data_type": [int, int],
            "block_size": [buffer_val_train_num, buffer_val_train_num] 
            }
        """
        
        validation_output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_val_train_num],
            "data_type": [int],
            "block_size": [buffer_val_train_num] 
            }
        
        
        #Create Generators
        print("Creating training generator...")
        #give training data num
        """training_generator = DGTrain.DataGeneratorTrain(input_buffer_params,
                                                           output_buffer_params,
                                                           [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        training_generator = DGTrain.DataGeneratorTrainErrorTypeEdition(input_buffer_params,
                                                                        output_buffer_params,
                                                                        [list(range(buffer_val_train_num))] 
                                                )
        #Create Generators
        print("Creating validation generator...")
        #give valitdation data num
        """validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params,
                                                                       validation_output_buffer_params,
                                                                      [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        validation_generator = DGValidation.DataGeneratorValidationErrorTypeEdition(validation_input_buffer_params,
                                                  validation_output_buffer_params,
                                                  [list(range(buffer_val_train_num))] 
                                                )
        
        
        #''' <-----traing switch
        #Start training
        print("Strat training...")
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit_generator(generator = training_generator,
                                      epochs = 1000, #100 200 500 3000
                                      verbose = 2, #set visibility
                                      validation_data = validation_generator,
                                      callbacks = [model_checkpoint_callback],
                                     )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [11]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [12]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [13]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [14]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [15]:
x=TestTranslate()
x.test_translate()

Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Creating training generator...
Creating validation generator...
Strat training...


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
21/21 - 8s - loss: 0.3121 - binary_accuracy: 0.9101 - val_loss: 0.2351 - val_binary_accuracy: 0.9549
Epoch 2/1000


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


21/21 - 3s - loss: 0.2169 - binary_accuracy: 0.9551 - val_loss: 0.2096 - val_binary_accuracy: 0.9549
Epoch 3/1000
21/21 - 3s - loss: 0.2046 - binary_accuracy: 0.9552 - val_loss: 0.2021 - val_binary_accuracy: 0.9549
Epoch 4/1000
21/21 - 3s - loss: 0.1980 - binary_accuracy: 0.9552 - val_loss: 0.1986 - val_binary_accuracy: 0.9549
Epoch 5/1000
21/21 - 3s - loss: 0.1945 - binary_accuracy: 0.9551 - val_loss: 0.1927 - val_binary_accuracy: 0.9550
Epoch 6/1000
21/21 - 3s - loss: 0.1897 - binary_accuracy: 0.9552 - val_loss: 0.1906 - val_binary_accuracy: 0.9549
Epoch 7/1000
21/21 - 3s - loss: 0.1882 - binary_accuracy: 0.9551 - val_loss: 0.1882 - val_binary_accuracy: 0.9549
Epoch 8/1000
21/21 - 3s - loss: 0.1859 - binary_accuracy: 0.9551 - val_loss: 0.1872 - val_binary_accuracy: 0.9549
Epoch 9/1000
21/21 - 3s - loss: 0.1851 - binary_accuracy: 0.9551 - val_loss: 0.1838 - val_binary_accuracy: 0.9550
Epoch 10/1000
21/21 - 3s - loss: 0.1837 - binary_accuracy: 0.9552 - val_loss: 0.1839 - val_binary_acc

...

Epoch 707/1000
21/21 - 3s - loss: 4.4218e-04 - binary_accuracy: 0.9999 - val_loss: 0.4934 - val_binary_accuracy: 0.9564
Epoch 708/1000
21/21 - 3s - loss: 3.8517e-04 - binary_accuracy: 0.9998 - val_loss: 0.4930 - val_binary_accuracy: 0.9561
Epoch 709/1000
21/21 - 3s - loss: 4.9193e-04 - binary_accuracy: 0.9998 - val_loss: 0.4931 - val_binary_accuracy: 0.9567
Epoch 710/1000
21/21 - 3s - loss: 4.8582e-04 - binary_accuracy: 0.9999 - val_loss: 0.4922 - val_binary_accuracy: 0.9568
Epoch 711/1000
21/21 - 3s - loss: 4.1144e-04 - binary_accuracy: 0.9999 - val_loss: 0.4943 - val_binary_accuracy: 0.9570
Epoch 712/1000
21/21 - 3s - loss: 4.1626e-04 - binary_accuracy: 0.9999 - val_loss: 0.4950 - val_binary_accuracy: 0.9563
Epoch 713/1000
21/21 - 3s - loss: 3.4940e-04 - binary_accuracy: 0.9999 - val_loss: 0.4967 - val_binary_accuracy: 0.9572
Epoch 714/1000
21/21 - 3s - loss: 3.6070e-04 - binary_accuracy: 0.9999 - val_loss: 0.4950 - val_binary_accuracy: 0.9563
Epoch 715/1000
21/21 - 3s - loss: 3.9092

Epoch 776/1000
21/21 - 3s - loss: 2.5535e-04 - binary_accuracy: 1.0000 - val_loss: 0.5173 - val_binary_accuracy: 0.9565
Epoch 777/1000
21/21 - 3s - loss: 2.3033e-04 - binary_accuracy: 1.0000 - val_loss: 0.5164 - val_binary_accuracy: 0.9565
Epoch 778/1000
21/21 - 3s - loss: 2.2555e-04 - binary_accuracy: 1.0000 - val_loss: 0.5176 - val_binary_accuracy: 0.9563
Epoch 779/1000
21/21 - 3s - loss: 2.9442e-04 - binary_accuracy: 0.9999 - val_loss: 0.5195 - val_binary_accuracy: 0.9564
Epoch 780/1000
21/21 - 3s - loss: 3.0209e-04 - binary_accuracy: 1.0000 - val_loss: 0.5182 - val_binary_accuracy: 0.9564
Epoch 781/1000
21/21 - 3s - loss: 2.8984e-04 - binary_accuracy: 0.9999 - val_loss: 0.5160 - val_binary_accuracy: 0.9561
Epoch 782/1000
21/21 - 3s - loss: 3.3094e-04 - binary_accuracy: 1.0000 - val_loss: 0.5162 - val_binary_accuracy: 0.9568
Epoch 783/1000
21/21 - 3s - loss: 2.8534e-04 - binary_accuracy: 1.0000 - val_loss: 0.5165 - val_binary_accuracy: 0.9567
Epoch 784/1000
21/21 - 3s - loss: 2.4776

Epoch 845/1000
21/21 - 3s - loss: 2.8497e-04 - binary_accuracy: 1.0000 - val_loss: 0.5343 - val_binary_accuracy: 0.9548
Epoch 846/1000
21/21 - 3s - loss: 7.9642e-04 - binary_accuracy: 0.9998 - val_loss: 0.5420 - val_binary_accuracy: 0.9559
Epoch 847/1000
21/21 - 3s - loss: 0.0021 - binary_accuracy: 0.9995 - val_loss: 0.5334 - val_binary_accuracy: 0.9533
Epoch 848/1000
21/21 - 3s - loss: 0.0019 - binary_accuracy: 0.9995 - val_loss: 0.5253 - val_binary_accuracy: 0.9525
Epoch 849/1000
21/21 - 3s - loss: 0.0028 - binary_accuracy: 0.9992 - val_loss: 0.5397 - val_binary_accuracy: 0.9541
Epoch 850/1000
21/21 - 3s - loss: 0.0023 - binary_accuracy: 0.9993 - val_loss: 0.5498 - val_binary_accuracy: 0.9549
Epoch 851/1000
21/21 - 3s - loss: 0.0024 - binary_accuracy: 0.9994 - val_loss: 0.5333 - val_binary_accuracy: 0.9550
Epoch 852/1000
21/21 - 3s - loss: 0.0012 - binary_accuracy: 0.9998 - val_loss: 0.5381 - val_binary_accuracy: 0.9561
Epoch 853/1000
21/21 - 3s - loss: 9.1633e-04 - binary_accuracy: 

Epoch 914/1000
21/21 - 3s - loss: 1.5682e-04 - binary_accuracy: 1.0000 - val_loss: 0.5593 - val_binary_accuracy: 0.9564
Epoch 915/1000
21/21 - 3s - loss: 2.2827e-04 - binary_accuracy: 0.9999 - val_loss: 0.5572 - val_binary_accuracy: 0.9562
Epoch 916/1000
21/21 - 3s - loss: 2.7639e-04 - binary_accuracy: 1.0000 - val_loss: 0.5567 - val_binary_accuracy: 0.9562
Epoch 917/1000
21/21 - 3s - loss: 2.0546e-04 - binary_accuracy: 1.0000 - val_loss: 0.5568 - val_binary_accuracy: 0.9563
Epoch 918/1000
21/21 - 3s - loss: 1.6677e-04 - binary_accuracy: 0.9999 - val_loss: 0.5578 - val_binary_accuracy: 0.9561
Epoch 919/1000
21/21 - 3s - loss: 2.1756e-04 - binary_accuracy: 1.0000 - val_loss: 0.5606 - val_binary_accuracy: 0.9563
Epoch 920/1000
21/21 - 3s - loss: 2.0100e-04 - binary_accuracy: 1.0000 - val_loss: 0.5604 - val_binary_accuracy: 0.9569
Epoch 921/1000
21/21 - 3s - loss: 1.9154e-04 - binary_accuracy: 1.0000 - val_loss: 0.5579 - val_binary_accuracy: 0.9567
Epoch 922/1000
21/21 - 3s - loss: 1.9426

Epoch 983/1000
21/21 - 3s - loss: 2.3018e-04 - binary_accuracy: 1.0000 - val_loss: 0.5747 - val_binary_accuracy: 0.9565
Epoch 984/1000
21/21 - 3s - loss: 2.4795e-04 - binary_accuracy: 0.9999 - val_loss: 0.5759 - val_binary_accuracy: 0.9561
Epoch 985/1000
21/21 - 3s - loss: 2.9035e-04 - binary_accuracy: 1.0000 - val_loss: 0.5771 - val_binary_accuracy: 0.9569
Epoch 986/1000
21/21 - 3s - loss: 1.8937e-04 - binary_accuracy: 1.0000 - val_loss: 0.5781 - val_binary_accuracy: 0.9571
Epoch 987/1000
21/21 - 3s - loss: 1.7668e-04 - binary_accuracy: 1.0000 - val_loss: 0.5744 - val_binary_accuracy: 0.9569
Epoch 988/1000
21/21 - 3s - loss: 1.6337e-04 - binary_accuracy: 1.0000 - val_loss: 0.5725 - val_binary_accuracy: 0.9569
Epoch 989/1000
21/21 - 3s - loss: 1.9369e-04 - binary_accuracy: 1.0000 - val_loss: 0.5752 - val_binary_accuracy: 0.9566
Epoch 990/1000
21/21 - 3s - loss: 1.6026e-04 - binary_accuracy: 1.0000 - val_loss: 0.5779 - val_binary_accuracy: 0.9569
Epoch 991/1000
21/21 - 3s - loss: 1.4484